# Using the shifted H clustering to improve color identification

## Function to generate the shifted H values

In [ ]:
h_shifts = []

Data must be in RGB format with each pixel as a seperate line.
Suggested v_thresholds and s_thresholds are 30% of sum of v and s

In [ ]:


def shift_h(data, v_thresh, s_thresh):
    """Produces shifted H values for color segmentation
    Inputs: data - list of pixel R,G,B values one entery per pixel
    Outputs: H, H120, H240
    """
    for i in range(0,len(data)):
        V = max(r,g,b)
        M = min(r,g,b)
        S = (V-M)/(1-(abs(V+M-1)))
        V_thres = V*v_thresh
        S_thres = V*s_thresh
        if V > v_thresh & S > s_thresh:
            R = 60*((g-b/V-M))
            G = 60*((2+(b-r))/(V-M))
            B = 60*((4 + (r-g))/(V-M))
            H = R+B+G
            if H >= 120:
                H120 = H - 120
            else:
                H120 = H + 60
            if H >= 60:
                H240 = H - 60
            else:
                H240 = H + 120
        else:
            H = 200 + ((V/255)*55)
            H120 = H
            H240 = H
        h_shifts.append[H,H120,H240]
            
    

# Run K++ on each H values

## Clustering & Bagging